In [6]:
import xml.dom.minidom
import uuid
import subprocess
import os
from tqdm import tqdm
import random 
import shutil 

In [20]:
# consts
WORKING_DIR = '/home/ax/data/DeepExplain'
DATA_DIR = f'{WORKING_DIR}/experiments/mitsuba/master_data'
SCENE_FILE_MASTER = f'{DATA_DIR}/master_mts.xml'

DATA_SET_ROOT = f'{WORKING_DIR}/experiments/data/allTransforms'

if not os.path.exists(DATA_SET_ROOT):
    os.mkdir(DATA_SET_ROOT)
    os.mkdir(f'{DATA_SET_ROOT}/train')
    os.mkdir(f'{DATA_SET_ROOT}/test')
    

master_doc = xml.dom.minidom.parse(SCENE_FILE_MASTER)


# number of samples
TRAIN_SAMPLES = 1800
TEST_SAMPLES = 200

# left and right
X_MIN = -100
X_MAX = 100

# depth
Z_MIN = 0

Z_MAX = -300

# rotation
ANGLE_MIN = -45
ANGLE_MAX = 45


POSE_ID_MIN = 1
POSE_ID_MAX = 11

master_requirements = [f'pose_0_{i}.serialized' for i in range(POSE_ID_MIN, POSE_ID_MAX)]
master_requirements.append(f'MikeAlger_Texture.jpeg')
master_requirements.append(f'rectangle.obj')
master_requirements.append(f'noise_texture.jpg')

for file in master_requirements:
    shutil.copy(f'{DATA_DIR}/{file}', f'{DATA_SET_ROOT}/train/{file}')
    shutil.copy(f'{DATA_DIR}/{file}', f'{DATA_SET_ROOT}/test/{file}')

In [21]:
# compute a scene string
def get_scene_xml_string(doc, x, z, angle, pose_id):
    shape_node_human = doc.getElementsByTagName('shape')[0]
    transform_node = shape_node_human.getElementsByTagName('transform')[0]
    translation_node = transform_node.getElementsByTagName('translate')[0]
    rotation_node = transform_node.getElementsByTagName('rotate')[0]
    filename_node = shape_node_human.getElementsByTagName('string')[0]
    
    translation_node.setAttribute('z', str(z)) 
    translation_node.setAttribute('x', str(x))
    rotation_node.setAttribute('angle', str(angle))
    filename_node.setAttribute('value', f"pose_0_{pose_id}.serialized")
    
    shape_node_rect = doc.getElementsByTagName('shape')[1]
    transform_node = shape_node_rect.getElementsByTagName('transform')[0]
    translation_node = transform_node.getElementsByTagName('translate')[1]
    rotation_node = transform_node.getElementsByTagName('rotate')[0]
    
    translation_node.setAttribute('z', str(z)) 
    translation_node.setAttribute('x', str(x))
    rotation_node.setAttribute('angle', str(angle))
    
    return doc.toxml()

# save the scene to file
def save_scene(scene_str, pose_id, path=DATA_SET_ROOT):
    file_name = f'{path}/a_{pose_id}_{uuid.uuid4()}.xml'
    with open(file_name, "w") as f:
        f.write(scene_str)
    return file_name


    
def generate_sample(master_doc, x, z, angle, pose_id, base_dir):
    # generate xml scene string
    scene_str = get_scene_xml_string(master_doc, x, z, angle, pose_id)
    # save scene as file
    scene_file_name = save_scene(scene_str, pose_id, base_dir)

In [22]:
print('Generate training data')
for i in tqdm(range(TRAIN_SAMPLES)):
    x = random.uniform(X_MIN, X_MAX)
    z = random.uniform(Z_MIN, Z_MAX)
    angle = random.uniform(ANGLE_MIN, ANGLE_MAX)
    pose_id = random.randint(POSE_ID_MIN, POSE_ID_MAX - 1)
    
    generate_sample(master_doc, x, z, angle, pose_id, f'{DATA_SET_ROOT}/train')
    
print('Generate test data')
for i in tqdm(range(TEST_SAMPLES)):
    x = random.uniform(X_MIN, X_MAX)
    z = random.uniform(Z_MIN, Z_MAX)
    angle = random.uniform(ANGLE_MIN, ANGLE_MAX)
    pose_id = random.randint(POSE_ID_MIN, POSE_ID_MAX - 1)
    
    generate_sample(master_doc, x, z, angle, pose_id, f'{DATA_SET_ROOT}/test')

 21%|██▏       | 385/1800 [00:00<00:00, 3842.55it/s]

Generate training data


100%|██████████| 200/200 [00:00<00:00, 3851.24it/s]

Generate test data
